In [1]:
%reset -f
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import plotly
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
from pandas_profiling import ProfileReport
pd.set_option('max_columns', 10000)
pd.set_option('max_rows', 10000)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
engine = create_engine("sqlite:////Users/nathanieldirenzo/Documents/GitHub/NBM_EDA_Student/resources/get_mta/mta_data.db")

In [3]:
df = pd.read_sql('SELECT * FROM mta_data LIMIT 100000;', engine)
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,03:00:00,REGULAR,7672966,2645023
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,07:00:00,REGULAR,7672969,2645040
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,11:00:00,REGULAR,7672991,2645127
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,15:00:00,REGULAR,7673037,2645193
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,19:00:00,REGULAR,7673097,2645270


In [4]:
#profile = ProfileReport(df)
#profile

In [5]:
df = df.rename(columns={'C/A': 'control_area', 'UNIT': 'unit', 'SCP': 'subunit_channel_pos', 'STATION':'station', 'LINENAME':'subway_lines', 'DIVISION':'division', 'DATE':'date', 'TIME':'time', 'DESC':'desc', 'ENTRIES':'entries', 'EXITS':'exits'})

In [6]:
df.head()

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,03:00:00,REGULAR,7672966,2645023
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,07:00:00,REGULAR,7672969,2645040
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,11:00:00,REGULAR,7672991,2645127
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,15:00:00,REGULAR,7673037,2645193
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,12/18/2021,19:00:00,REGULAR,7673097,2645270


In [7]:
df['date'] = df['date'].str.replace('/','_')
df['time'] = df['time'].str.replace(':','_')
df['desc'] = df['desc'].str.replace(' ', '_')
df['subunit_channel_pos'] = df['subunit_channel_pos'].str.replace('-', '_')
df['station'] = df['station'].str.replace('-', '_')
df['station'] = df['station'].str.replace(' ', '_')
df['station'] = df['station'].str.replace('/','_')

In [8]:
df['unique_id'] = df['control_area'] + '_' + df['unit'] + '_' + df['subunit_channel_pos'] + '_' + df['station'] + '_' + df['date'] + '_' + df['time'] + '_' + df['desc']

In [9]:
df = df.set_index('unique_id')

In [10]:
df['date_time'] = df.date + ' ' + df.time

In [11]:
df.date_time = pd.to_datetime(df['date_time'], format = '%m_%d_%Y %H_%M_%S')

In [12]:
df.sample(100)

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time
unique_id,,,,,,,,,,,,
N414_R316_00_00_00_FLUSHING_AV_12_23_2021_16_00_00_REGULAR,N414,R316,00_00_00,FLUSHING_AV,G,IND,12_23_2021,16_00_00,REGULAR,662534,1841116,2021-12-23 16:00:00
B026_R230_00_00_03_NECK_RD_12_20_2021_07_00_00_REGULAR,B026,R230,00_00_03,NECK_RD,BQ,BMT,12_20_2021,07_00_00,REGULAR,328833,75102,2021-12-20 07:00:00
N135_R385_01_03_00_ROCKAWAY_BLVD_12_18_2021_16_00_00_REGULAR,N135,R385,01_03_00,ROCKAWAY_BLVD,A,IND,12_18_2021,16_00_00,REGULAR,136082698,799705,2021-12-18 16:00:00
J028_R004_00_00_04_75_ST_ELDERTS_12_24_2021_11_00_00_REGULAR,J028,R004,00_00_04,75_ST_ELDERTS,JZ,BMT,12_24_2021,11_00_00,REGULAR,251758,199431,2021-12-24 11:00:00
N020_R101_00_00_01_145_ST_12_24_2021_23_00_00_REGULAR,N020,R101,00_00_01,145_ST,ABCD,IND,12_24_2021,23_00_00,REGULAR,105261,136640,2021-12-24 23:00:00
A006_R079_00_00_02_5_AV_59_ST_12_21_2021_13_00_00_REGULAR,A006,R079,00_00_02,5_AV_59_ST,NQRW,BMT,12_21_2021,13_00_00,REGULAR,6620632,16635664,2021-12-21 13:00:00
H007A_R248_02_05_01_1_AV_12_24_2021_11_00_00_REGULAR,H007A,R248,02_05_01,1_AV,L,BMT,12_24_2021,11_00_00,REGULAR,524311,184,2021-12-24 11:00:00
N422_R318_00_00_01_FULTON_ST_12_21_2021_19_00_00_REGULAR,N422,R318,00_00_01,FULTON_ST,G,IND,12_21_2021,19_00_00,REGULAR,5798832,5295406,2021-12-21 19:00:00
N206_R104_01_00_01_167_ST_12_19_2021_19_00_00_REGULAR,N206,R104,01_00_01,167_ST,BD,IND,12_19_2021,19_00_00,REGULAR,325138,99573,2021-12-19 19:00:00


In [13]:
df = df[df.desc != 'RECOVR_AUD']

In [14]:
df['net_entries'] = df.groupby(['control_area', 'unit', 'subunit_channel_pos', 'station'])['entries'].transform(lambda x: x.diff())
df['net_exits'] = df.groupby(['control_area', 'unit', 'subunit_channel_pos', 'station'])['exits'].transform(lambda x: x.diff())
df['net_traffic'] = df.net_entries + df.net_exits

In [15]:
df.head(20)

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time,net_entries,net_exits,net_traffic
unique_id,,,,,,,,,,,,,,,
A002_R051_02_00_00_59_ST_12_18_2021_03_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,03_00_00,REGULAR,7672966,2645023,2021-12-18 03:00:00,NaN,NaN,NaN
A002_R051_02_00_00_59_ST_12_18_2021_07_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,07_00_00,REGULAR,7672969,2645040,2021-12-18 07:00:00,3.0,17.0,20.0
A002_R051_02_00_00_59_ST_12_18_2021_11_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,11_00_00,REGULAR,7672991,2645127,2021-12-18 11:00:00,22.0,87.0,109.0
A002_R051_02_00_00_59_ST_12_18_2021_15_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,15_00_00,REGULAR,7673037,2645193,2021-12-18 15:00:00,46.0,66.0,112.0
A002_R051_02_00_00_59_ST_12_18_2021_19_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,19_00_00,REGULAR,7673097,2645270,2021-12-18 19:00:00,60.0,77.0,137.0
A002_R051_02_00_00_59_ST_12_18_2021_23_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,23_00_00,REGULAR,7673128,2645283,2021-12-18 23:00:00,31.0,13.0,44.0
A002_R051_02_00_00_59_ST_12_19_2021_03_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,03_00_00,REGULAR,7673141,2645289,2021-12-19 03:00:00,13.0,6.0,19.0
A002_R051_02_00_00_59_ST_12_19_2021_07_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,07_00_00,REGULAR,7673146,2645299,2021-12-19 07:00:00,5.0,10.0,15.0
A002_R051_02_00_00_59_ST_12_19_2021_11_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,11_00_00,REGULAR,7673153,2645360,2021-12-19 11:00:00,7.0,61.0,68.0


In [16]:
df[df.net_traffic < 0][['net_traffic', 'net_entries', 'net_exits']].sort_values(by='net_traffic')

,net_traffic,net_entries,net_exits
unique_id,,,
A050_R088_00_03_02_CORTLANDT_ST_12_21_2021_16_00_00_REGULAR,-42183.0,-28412.0,-13771.0
N056_R188_01_00_03_50_ST_12_20_2021_19_00_00_REGULAR,-908.0,-340.0,-568.0
N056_R188_01_00_03_50_ST_12_21_2021_19_00_00_REGULAR,-861.0,-287.0,-574.0
H007A_R248_02_03_00_1_AV_12_20_2021_19_00_00_REGULAR,-804.0,-99.0,-705.0
H007A_R248_02_03_00_1_AV_12_18_2021_19_00_00_REGULAR,-784.0,-53.0,-731.0
N056_R188_01_00_03_50_ST_12_22_2021_19_00_00_REGULAR,-761.0,-250.0,-511.0
H007A_R248_02_03_00_1_AV_12_22_2021_19_00_00_REGULAR,-747.0,-85.0,-662.0
H007A_R248_02_03_00_1_AV_12_19_2021_19_00_00_REGULAR,-712.0,-47.0,-665.0
N062_R011_01_00_01_42_ST_PORT_AUTH_12_21_2021_19_00_00_REGULAR,-701.0,65.0,-766.0


In [17]:
df['net_entries'] = abs(df.net_entries)
df['net_exits'] = abs(df.net_exits)
df['net_traffic'] = abs(df.net_traffic)

In [18]:
df.net_entries.isna().sum()

2373

In [19]:
q = np.nanquantile(df["net_entries"], .99)
df = df[df["net_entries"] < q]

q2 = np.nanquantile(df["net_exits"], .99)
df = df[df["net_exits"] < q2]

q3 = np.nanquantile(df['net_traffic'], .99)
df=df[df['net_traffic'] < q3]

In [20]:
df.net_entries.isna().sum()

0

In [21]:
df.head(20)

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time,net_entries,net_exits,net_traffic
unique_id,,,,,,,,,,,,,,,
A002_R051_02_00_00_59_ST_12_18_2021_07_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,07_00_00,REGULAR,7672969,2645040,2021-12-18 07:00:00,3.0,17.0,20.0
A002_R051_02_00_00_59_ST_12_18_2021_11_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,11_00_00,REGULAR,7672991,2645127,2021-12-18 11:00:00,22.0,87.0,109.0
A002_R051_02_00_00_59_ST_12_18_2021_15_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,15_00_00,REGULAR,7673037,2645193,2021-12-18 15:00:00,46.0,66.0,112.0
A002_R051_02_00_00_59_ST_12_18_2021_19_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,19_00_00,REGULAR,7673097,2645270,2021-12-18 19:00:00,60.0,77.0,137.0
A002_R051_02_00_00_59_ST_12_18_2021_23_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,23_00_00,REGULAR,7673128,2645283,2021-12-18 23:00:00,31.0,13.0,44.0
A002_R051_02_00_00_59_ST_12_19_2021_03_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,03_00_00,REGULAR,7673141,2645289,2021-12-19 03:00:00,13.0,6.0,19.0
A002_R051_02_00_00_59_ST_12_19_2021_07_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,07_00_00,REGULAR,7673146,2645299,2021-12-19 07:00:00,5.0,10.0,15.0
A002_R051_02_00_00_59_ST_12_19_2021_11_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,11_00_00,REGULAR,7673153,2645360,2021-12-19 11:00:00,7.0,61.0,68.0
A002_R051_02_00_00_59_ST_12_19_2021_15_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,15_00_00,REGULAR,7673179,2645407,2021-12-19 15:00:00,26.0,47.0,73.0


In [22]:
df['audit_length_hours'] = df.groupby(['control_area', 'unit', 'subunit_channel_pos', 'station'])['date_time'].transform(lambda x: x.diff())

In [23]:
df['audit_length_hours'] = df.audit_length_hours.dt.total_seconds() / 3600
#df.audit_length_hours = df.audit_length_hours.fillna(df.audit_length_hours.mean())

In [24]:
df.audit_length_hours.value_counts()

4.000000     89173
8.000000      1373
12.000000      376
16.000000      264
0.022222       168
0.022500       113
20.000000       79
0.021944        24
2.477778        19
0.026667        19
1.383889        18
3.270278        18
0.640556        17
0.643889        10
0.044444        10
3.311111        10
0.220833        10
0.021667        10
0.178889         8
2.138056         8
1.683056         6
3.751389         6
0.027778         6
1.756667         6
2.177222         6
3.512500         6
0.487500         6
3.750000         5
1.752222         5
3.779167         4
3.258333         4
0.023056         4
86.840556        4
9.159444         4
6.780556         4
0.908056         4
24.000000        4
0.607778         3
32.000000        3
1.679167         3
2.320833         3
1.311111         3
2.688889         3
0.044722         3
0.066667         3
5.683056         2
10.163889        2
0.089167         2
0.685000         1
2.024444         1
15.270278        1
6.186389         1
4.607778    

In [25]:
df = df[df.audit_length_hours == 4.0]

In [26]:
df.head(20)

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time,net_entries,net_exits,net_traffic,audit_length_hours
unique_id,,,,,,,,,,,,,,,,
A002_R051_02_00_00_59_ST_12_18_2021_11_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,11_00_00,REGULAR,7672991,2645127,2021-12-18 11:00:00,22.0,87.0,109.0,4.0
A002_R051_02_00_00_59_ST_12_18_2021_15_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,15_00_00,REGULAR,7673037,2645193,2021-12-18 15:00:00,46.0,66.0,112.0,4.0
A002_R051_02_00_00_59_ST_12_18_2021_19_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,19_00_00,REGULAR,7673097,2645270,2021-12-18 19:00:00,60.0,77.0,137.0,4.0
A002_R051_02_00_00_59_ST_12_18_2021_23_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,23_00_00,REGULAR,7673128,2645283,2021-12-18 23:00:00,31.0,13.0,44.0,4.0
A002_R051_02_00_00_59_ST_12_19_2021_03_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,03_00_00,REGULAR,7673141,2645289,2021-12-19 03:00:00,13.0,6.0,19.0,4.0
A002_R051_02_00_00_59_ST_12_19_2021_07_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,07_00_00,REGULAR,7673146,2645299,2021-12-19 07:00:00,5.0,10.0,15.0,4.0
A002_R051_02_00_00_59_ST_12_19_2021_11_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,11_00_00,REGULAR,7673153,2645360,2021-12-19 11:00:00,7.0,61.0,68.0,4.0
A002_R051_02_00_00_59_ST_12_19_2021_15_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,15_00_00,REGULAR,7673179,2645407,2021-12-19 15:00:00,26.0,47.0,73.0,4.0
A002_R051_02_00_00_59_ST_12_19_2021_19_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,19_00_00,REGULAR,7673218,2645445,2021-12-19 19:00:00,39.0,38.0,77.0,4.0


In [27]:
df.dtypes

control_area                   object
unit                           object
subunit_channel_pos            object
station                        object
subway_lines                   object
division                       object
date                           object
time                           object
desc                           object
entries                         int64
exits                           int64
date_time              datetime64[ns]
net_entries                   float64
net_exits                     float64
net_traffic                   float64
audit_length_hours            float64
dtype: object

In [29]:
df[['pct_change_entries', 'pct_change_exits', 'pct_change_traffic']].isna().sum()

pct_change_entries    8740
pct_change_exits      9598
pct_change_traffic    5838
dtype: int64

In [30]:
df.dropna(inplace=True)

In [31]:
df['day_of_week'] = df.date_time.dt.dayofweek

In [32]:
df.head()

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time,net_entries,net_exits,net_traffic,audit_length_hours,pct_change_entries,pct_change_exits,pct_change_traffic,day_of_week
unique_id,,,,,,,,,,,,,,,,,,,,
A002_R051_02_00_00_59_ST_12_18_2021_15_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,15_00_00,REGULAR,7673037,2645193,2021-12-18 15:00:00,46.0,66.0,112.0,4.0,109.090909,-24.137931,2.752294,5
A002_R051_02_00_00_59_ST_12_18_2021_19_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,19_00_00,REGULAR,7673097,2645270,2021-12-18 19:00:00,60.0,77.0,137.0,4.0,30.434783,16.666667,22.321429,5
A002_R051_02_00_00_59_ST_12_18_2021_23_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_18_2021,23_00_00,REGULAR,7673128,2645283,2021-12-18 23:00:00,31.0,13.0,44.0,4.0,-48.333333,-83.116883,-67.883212,5
A002_R051_02_00_00_59_ST_12_19_2021_03_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,03_00_00,REGULAR,7673141,2645289,2021-12-19 03:00:00,13.0,6.0,19.0,4.0,-58.064516,-53.846154,-56.818182,6
A002_R051_02_00_00_59_ST_12_19_2021_07_00_00_REGULAR,A002,R051,02_00_00,59_ST,NQR456W,BMT,12_19_2021,07_00_00,REGULAR,7673146,2645299,2021-12-19 07:00:00,5.0,10.0,15.0,4.0,-61.538462,66.666667,-21.052632,6


In [33]:
df = df.drop(labels=['control_area', 'unit', 'subunit_channel_pos', 'division', 'desc'], axis=1)

In [34]:
df.head()

,station,subway_lines,date,time,entries,exits,date_time,net_entries,net_exits,net_traffic,audit_length_hours,pct_change_entries,pct_change_exits,pct_change_traffic,day_of_week
unique_id,,,,,,,,,,,,,,,
A002_R051_02_00_00_59_ST_12_18_2021_15_00_00_REGULAR,59_ST,NQR456W,12_18_2021,15_00_00,7673037,2645193,2021-12-18 15:00:00,46.0,66.0,112.0,4.0,109.090909,-24.137931,2.752294,5
A002_R051_02_00_00_59_ST_12_18_2021_19_00_00_REGULAR,59_ST,NQR456W,12_18_2021,19_00_00,7673097,2645270,2021-12-18 19:00:00,60.0,77.0,137.0,4.0,30.434783,16.666667,22.321429,5
A002_R051_02_00_00_59_ST_12_18_2021_23_00_00_REGULAR,59_ST,NQR456W,12_18_2021,23_00_00,7673128,2645283,2021-12-18 23:00:00,31.0,13.0,44.0,4.0,-48.333333,-83.116883,-67.883212,5
A002_R051_02_00_00_59_ST_12_19_2021_03_00_00_REGULAR,59_ST,NQR456W,12_19_2021,03_00_00,7673141,2645289,2021-12-19 03:00:00,13.0,6.0,19.0,4.0,-58.064516,-53.846154,-56.818182,6
A002_R051_02_00_00_59_ST_12_19_2021_07_00_00_REGULAR,59_ST,NQR456W,12_19_2021,07_00_00,7673146,2645299,2021-12-19 07:00:00,5.0,10.0,15.0,4.0,-61.538462,66.666667,-21.052632,6


In [51]:
station_daily = df.groupby(["station", pd.Grouper(key='date_time', freq='D')])[['net_traffic']].sum().sort_values(by=['station', 'date_time']).reset_index()
station_daily

,station,date_time,net_traffic
0,103_ST,2021-12-18,1984.0
1,103_ST,2021-12-19,2481.0
2,103_ST,2021-12-20,3740.0
3,103_ST,2021-12-21,3581.0
4,103_ST,2021-12-22,3660.0
5,103_ST,2021-12-23,3358.0
6,103_ST,2021-12-24,2356.0
7,104_ST,2021-12-18,1643.0
8,104_ST,2021-12-19,1815.0
9,104_ST,2021-12-20,4067.0


In [53]:
station_daily['pct_change'] = station_daily.net_traffic.pct_change() * 100
station_daily

,station,date_time,net_traffic,pct_change
0,103_ST,2021-12-18,1984.0,NaN
1,103_ST,2021-12-19,2481.0,25.050403
2,103_ST,2021-12-20,3740.0,50.745667
3,103_ST,2021-12-21,3581.0,-4.251337
4,103_ST,2021-12-22,3660.0,2.206088
5,103_ST,2021-12-23,3358.0,-8.251366
6,103_ST,2021-12-24,2356.0,-29.839190
7,104_ST,2021-12-18,1643.0,-30.263158
8,104_ST,2021-12-19,1815.0,10.468655
9,104_ST,2021-12-20,4067.0,124.077135


In [71]:
station_yearly = station_daily.groupby(['station', pd.Grouper(key='date_time', freq='Y')])[['station', 'date_time','net_traffic']].sum().sort_values(by = 'net_traffic', ascending=False).reset_index()
station_yearly['pct_total'] = station_yearly.net_traffic / station_yearly.net_traffic.sum()
station_yearly.sort_values('pct_total', ascending=False)

,station,date_time,net_traffic,pct_total
0,34_ST_PENN_STA,2021-12-31,303127.0,0.031878
1,42_ST_PORT_AUTH,2021-12-31,266146.0,0.027989
2,FULTON_ST,2021-12-31,232891.0,0.024492
3,CANAL_ST,2021-12-31,226945.0,0.023867
4,8_AV,2021-12-31,148800.0,0.015649
5,47_50_STS_ROCK,2021-12-31,144994.0,0.015248
6,23_ST,2021-12-31,143043.0,0.015043
7,JAY_ST_METROTEC,2021-12-31,135584.0,0.014259
8,59_ST_COLUMBUS,2021-12-31,131131.0,0.013790
9,14_ST_UNION_SQ,2021-12-31,126553.0,0.013309


In [ ]:
#profile = ProfileReport(df, title="Pandas Profiling Report")
#profile

In [ ]:
df.station.nunique()

In [ ]:
df.groupby('station')['net_traffic'].mean().sort_values(ascending=False)

In [ ]:
nta_map = gpd.read_file('/Users/nathanieldirenzo/Documents/GitHub/NYC_MTA_EDA_Project/nynta2020.shp')
nta_map.head()
nta_map.to_crs(4326, inplace=True)
nta_map.crs

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
nta_map.plot(ax=ax);

In [ ]:
station_df = gpd.read_file('Subway Stations.geojson')
len(station_df)

In [ ]:
df.station.value_counts()

In [ ]:
station_df.name = station_df.name.str.replace(' - ', '_')
station_df.name = station_df.name.str.replace(' ', '_')
station_df.name = station_df.name.str.replace('(', '')
station_df.name = station_df.name.str.replace(')', '')
station_df.name = station_df.name.str.replace('/', '')
station_df.name = station_df.name.str.replace('', '')
station_df


In [ ]:
station_df.to_crs(4326, inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(10,10))
station_df.plot(ax=ax, markersize=20, color='blue', marker='o', label = 'Station')

In [ ]:
fig,ax = plt.subplots(figsize=(20,20))
nta_map.plot(ax=ax,);
station_df.plot(ax=ax, markersize=20, color='red', marker='o', label = 'Station');